### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 2x faster
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b 2x faster!
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
    "unsloth/gemma-2-9b-bnb-4bit",  # Gemma 2x faster!
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"  # Qwen 2.5 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",  # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.3: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

tokenizer_config.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.15M [00:00<?, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.3 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Phi-4` format for conversation style finetunes. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Phi-4 renders multi turn conversations like below:

```
<|im_start|>user<|im_sep|>Hello!<|im_end|>
<|im_start|>assistant<|im_sep|>Hi! How can I help?<|im_end|>
<|im_start|>user<|im_sep|>What is 2+2?<|im_end|>
```


In [ ]:
data = [{"prompt": "<|user|>\nWhat is Microsoft Copilot Studio?\n<|assistant|>\nMicrosoft Copilot Studio is a low-code tool for customizing and extending Microsoft Copilot experiences. It allows users to create custom copilots, integrate with APIs, and connect with enterprise data sources."},

{"prompt": "<|user|>\nWhat capabilities does Copilot Studio offer?\n<|assistant|>\nCopilot Studio offers capabilities such as topic creation, generative answers, Power Automate integration, entity extraction, conversation boosters, and plugin support."},

{"prompt": "<|user|>\nWhat are conversation boosters in Copilot Studio?\n<|assistant|>\nConversation boosters are a feature in Copilot Studio that allow you to connect your copilot with other knowledge sources like SharePoint, OneDrive, and Dataverse to enhance its ability to answer user questions."},

{"prompt": "<|user|>\nHow can users customize Copilot behavior?\n<|assistant|>\nUsers can customize Copilot behavior by editing topics, adding triggers, changing system instructions, and using Power Automate flows or plugins."},

{"prompt": "<|user|>\nWhat are Generative Answers?\n<|assistant|>\nGenerative Answers in Copilot Studio use Azure OpenAI Service to generate responses based on external knowledge sources connected through conversation boosters."},

{"prompt": "<|user|>\nWhat is a plugin in Copilot Studio?\n<|assistant|>\nA plugin is a reusable component that connects Copilot to an external service using APIs or Power Automate, allowing it to perform actions like booking, searching, or updating records."},

{"prompt": "<|user|>\nHow are plugins different from Power Automate?\n<|assistant|>\nPlugins are better for synchronous operations and reuse, while Power Automate is more suitable for asynchronous or multi-step processes. Both can be used to extend Copilot capabilities."},

{"prompt": "<|user|>\nCan I use Copilot Studio with other Microsoft Copilots?\n<|assistant|>\nYes, Copilot Studio can be used to extend and customize Copilots across Microsoft 365 apps and services such as Teams, Power Apps, and Dynamics 365."},

{"prompt": "<|user|>\nHow is Copilot Studio licensed?\n<|assistant|>\nCopilot Studio is licensed separately and also included in some Microsoft 365 plans. Licensing includes usage limits for messages, generative answers, and plugin/API calls."},

{"prompt": "<|user|>\nIs there a trial version of Copilot Studio?\n<|assistant|>\nYes, a 60-day free trial of Copilot Studio is available through the Microsoft 365 admin center or the Power Platform admin portal."},

{"prompt": "<|user|>\nWhat are the message usage limits in Copilot Studio?\n<|assistant|>\nEach licensed user gets up to 25,000 messages per month. Additional capacity can be purchased as needed."},

{"prompt": "<|user|>\nAre there specific environments required to use Copilot Studio?\n<|assistant|>\nYes, Copilot Studio requires environments with Dataverse and specific language support. You can manage environments through the Power Platform admin center."},

{"prompt": "<|user|>\nWhat languages are supported in Copilot Studio?\n<|assistant|>\nCopilot Studio supports over 100 languages for end-user conversations, but customization and configuration features are currently available only in English."},

{"prompt": "<|user|>\nWhat kind of analytics does Copilot Studio provide?\n<|assistant|>\nCopilot Studio provides usage analytics such as session volume, resolution rate, escalation rate, and topic triggering frequency to help you improve your copilots."},

{"prompt": "<|user|>\nCan I monitor and debug Copilot conversations?\n<|assistant|>\nYes, Copilot Studio offers tools like session transcripts, topic triggering logs, and runtime tracing to monitor and debug bot conversations."},

]

In [ ]:
def convert_prompt_to_conversation(data):
    converted_data = []
    for item in data:
        prompt = item["prompt"]
        user_split = prompt.split("<|assistant|>")[0].replace("<|user|>", "").strip()
        assistant_split = prompt.split("<|assistant|>")[1].strip()

        convo = [
            {"role": "user", "content": user_split},
            {"role": "assistant", "content": assistant_split}
        ]
        converted_data.append({"conversations": convo})
    return converted_data


In [ ]:
from datasets import Dataset
from unsloth.chat_templates import get_chat_template

# Verini yükle
raw_data = convert_prompt_to_conversation(data)

# HuggingFace Dataset'e çevir
dataset = Dataset.from_list(raw_data)

# Phi-4 tokenizer formatını al
tokenizer = get_chat_template(tokenizer, chat_template="phi-4")

# Formatting fonksiyonu
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=False
        )
        for convo in convos
    ]
    return { "text": texts }

# Dataset'i formatla
formatted_dataset = dataset.map(formatting_prompts_func, batched=False)


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
formatted_dataset[3]["conversations"]

[{'content': 'How can users customize Copilot behavior?', 'role': 'user'},
 {'content': 'Users can customize Copilot behavior by editing topics, adding triggers, changing system instructions, and using Power Automate flows or plugins.',
  'role': 'assistant'}]

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",  # phi-4 formatı: <|im_start|>user<|im_sep|>Hello<|im_end|>...
)

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False  # True olursa assistant cevabını model tamamlamaya çalışır
        )
        for convo in convos
    ]
    return { "text": texts }


In [ ]:
dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
dataset[0]["text"]

'<|im_start|>user<|im_sep|>What is Microsoft Copilot Studio?<|im_end|><|im_start|>assistant<|im_sep|>Microsoft Copilot Studio is a low-code tool for customizing and extending Microsoft Copilot experiences. It allows users to create custom copilots, integrate with APIs, and connect with enterprise data sources.<|im_end|>'

<a name="Train"></a>
### Train the model


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>user<|im_sep|>What is a plugin in Copilot Studio?<|im_end|><|im_start|>assistant<|im_sep|>A plugin is a reusable component that connects Copilot to an external service using APIs or Power Automate, allowing it to perform actions like booking, searching, or updating records.<|im_end|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                A plugin is a reusable component that connects Copilot to an external service using APIs or Power Automate, allowing it to perform actions like booking, searching, or updating records.<|im_end|>'

We can see the System and Instruction prompts are successfully masked!

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.727 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15 | Num Epochs = 15 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 65,536,000/4,000,000,000 (1.64% trained)


Step,Training Loss
1,3.260200
2,3.138700
3,3.228300
4,2.857600
5,2.427600
6,2.315200
7,1.602500
8,1.706800
9,1.350800
10,1.241700


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

191.7302 seconds used for training.
3.2 minutes used for training.
Peak reserved memory = 10.93 GB.
Peak reserved memory for training = 0.203 GB.
Peak reserved memory % of max memory = 74.147 %.
Peak reserved memory for training % of max memory = 1.377 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

We use `min_p = 0.1` and `temperature = 1.5`.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = formatted_dataset[3]["conversations"]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1
)
tokenizer.batch_decode(outputs)

['<|im_start|>user<|im_sep|>How can users customize Copilot behavior?<|im_end|><|im_start|>assistant<|im_sep|>Users can customize Copilot behavior by editing topics, adding triggers, changing system instructions, and using Power Automate flows or plugins.<|im_end|><|im_start|>assistant<|im_sep|>Great! You can edit topics, add triggers, change system instructions, and use Power Automate flows or plugins to customize Copilot behavior.<|im_end|>']

 You can also use a `TextStreamer` for continuous inference - so you can see The generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = formatted_dataset[3]["conversations"]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    return_attention_mask=True
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

Great! You can edit topics, add triggers, change system instructions, and use Power Automate flows or plugins to customize Copilot behavior.<|im_end|>


## 🧪 Inference on Custom Dataset and Exporting Model Outputs to CSV

This section performs **inference using a fine-tuned language model** (e.g., Phi-4 with LoRA adapters) on a custom dataset containing chat-style prompts and expected responses. The model's outputs are collected and saved into a CSV file for further analysis.

### 📝 Dataset Format
Each entry in the dataset is a single string with special formatting using tokens:

<|im_start|>user<|im_sep|>QUESTION<|im_end|>

<|im_start|>assistant<|im_sep|>EXPECTED_ANSWER<|im_end|>



### 🎯 Goal
- Extract the `QUESTION` part from each dataset item.
- Generate a response using the model.
- Compare it with the `EXPECTED_ANSWER` (optional).
- Save all results into a `model_outputs.csv` file.

### 📤 Output CSV Columns
- **question**: User input extracted from the dataset.
- **expected_answer**: The reference assistant answer.
- **generated_answer**: The model’s generated response.

This step is essential for validating how well the model performs on custom examples and helps in error analysis or evaluation.


In [ ]:
import torch
import openpyxl
from transformers import TextStreamer
from openpyxl import Workbook

results = []

for i, row in enumerate(dataset):
    text = row["text"]

    # Soruyu çıkar
    try:
        question = text.split("<|im_sep|>")[1].split("<|im_end|>")[0].strip()
    except:
        print(f"Parse hatası satır {i}: {text}")
        continue

    messages = [{"role": "user", "content": question}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # Modelden yanıt al
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            use_cache=True,
        )

    # Tokenleri çöz
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Sadece assistant cevabını al
    if "<|im_start|>assistant<|im_sep|>" in decoded_output:
        generated_answer = decoded_output.split("<|im_start|>assistant<|im_sep|>")[-1].split("<|im_end|>")[0].strip()
    else:
        generated_answer = decoded_output.strip()

    # Referans cevabı al
    try:
        expected_answer = text.split("<|im_start|>assistant<|im_sep|>")[1].split("<|im_end|>")[0].strip()
    except:
        expected_answer = ""

    results.append((question, expected_answer, generated_answer))


wb = Workbook()
ws = wb.active
ws.title = "Model Cevapları"

ws.append(["question", "expected_answer", "generated_answer"])

# Datas
for row in results:
    ws.append(row)

wb.save("model_outputs.xlsx")
print("✅ Excel dosyası 'model_outputs.xlsx' olarak kaydedildi.")


✅ Excel dosyası 'model_outputs.xlsx' olarak kaydedildi.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How can users customize Copilot behavior?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

Users can customize Copilot behavior by editing topics, adding triggers, changing system instructions, and using Power Automate flows or plugins.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
from google.colab import userdata
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list in our [Docs](https://docs.unsloth.ai/basics/saving-and-using-models/saving-to-gguf)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )